In [1]:
#@title Install, Import Libraries
%%capture
!pip install qiskit==1.0.1
!pip install qiskit_algorithms==0.3.0
!pip install qiskit_nature==0.7.2
!pip install --prefer-binary pyscf
!pip install openfermion
!pip install openfermionpyscf

import numpy as np
from qiskit import *
from pyscf import *
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator
import matplotlib.pyplot as plt
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.ops.representations import get_tensors_from_integrals
from openfermionpyscf import run_pyscf

In [2]:
def esProblem_H4(R = 0.7):
    ''' Returns the electronic structure problem of H4 each separated by R'''
    driver = PySCFDriver(
        atom = f"H 0 0 0; H 0 0 {R}; H 0 0 {2*R}; H 0 0 {3*R}",
        basis = "sto3g",
        charge = 0,
        spin = 0,
        unit = DistanceUnit.ANGSTROM,
    )
    es_problem = driver.run()
    return es_problem

def fci_h4(R):
    ''' Returns the FCI ground state energy (electronic + nuclear repulsion)
    based on the distance between H4 chain'''
    # Define parameters
    basis = "sto-3g"
    multiplicity = 1
    charge = 0
    # XYZ coordinates
    geometry = [("H", (0, 0, 0)), \
                ("H", (0, 0, R)), \
                ("H", (0, 0, 2*R)), \
                ("H", (0, 0, 3*R))]
    # Define molecule
    molecule = MolecularData(geometry, basis, multiplicity, charge)
    # Run HF calculation
    molecule = run_pyscf(molecule, run_fci = 1)
    return molecule.hf_energy, molecule.fci_energy

In [3]:
def qiskit_uccsd(es_problem):
    '''Returns the ground state energy (electronic + nuclear repulsion) from
    a Qiskit qubit-based UCCSD ansatz, based on the electronic structure problem'''
    # Use the Jordan-Wigner mapping
    mapper = JordanWignerMapper()
    # Initialize VQE solver: Hartree-Fock initial state, UCCSD variational form
    ansatz = UCCSD(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
        initial_state = HartreeFock(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
        ),
    )
    vqe_solver = VQE(Estimator(), ansatz, L_BFGS_B())
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters
    calc = GroundStateEigensolver(mapper, vqe_solver)
    res = calc.solve(es_problem)
    return res.total_energies[0]

In [ ]:
R_H4 = np.linspace(0.5, 2.5, num = 21)
fci_energies_H4 = []
qiskit_uccsd_energies_H4 = []
for r in R_H4:
    mol = esProblem_H4(r)
    qiskit_uccsd_energy = qiskit_uccsd(mol)
    print(f"R={r}, Energy={qiskit_uccsd_energy}")
    qiskit_uccsd_energies_H4.append(qiskit_uccsd_energy)
    fci_energies_H4.append(fci_h4(r)[1])

In [ ]:
print(np.average(np.array(qiskit_uccsd_energies_H4) - np.array(fci_energies_H4)))